In [1]:
from quelfilm import utils
from quelfilm.nlp.load import MdLoader
from quelfilm.nlp.preprocessing import Preprocessing
from quelfilm.nlp.representation import MergedMatrixRepresentation
from quelfilm.nlp.classifier import NaiveBayseClassifier, ClassificationProcessor, NaiveBayseTfIdfClassifier

In [13]:
loader = MdLoader('./training.md')
processor = Preprocessing(loader)
all_data = processor.data

./training.md


[nltk_data] Error loading french: Package 'french' not found in index


In [81]:
import numpy as np
import typing as t


class DataGroup:
    test: t.Dict[str, np.array]
    train: t.Dict[str, np.array]


def get_spliting_idx(splits: int, array: np.array) -> np.array:
    length = len(array)
    elements = length // splits
    rest = length % splits
    split_idx = np.full(splits, elements)
    split_idx[0] = split_idx[0] + rest
    return split_idx


def get_masks(splits: int, array: np.array) -> t.List[np.array]:
    split_idx: np.array = get_spliting_idx(splits, array)
    masks = []
    for i in range(len(split_idx)):
        mask = np.zeros(len(array))
        if i == 0:
            ones = np.ones(split_idx[i])
            mask = np.concatenate((ones, np.zeros(len(array) - len(ones))))
        else:
            for j in range(sum(split_idx[:i]), sum(split_idx[:i+1])):
                mask[j] = 1
        mask = mask == 1
        masks.append(mask)
    return masks


def convert_to_numpy(data: t.Dict[str, t.List[t.List]]) -> t.Dict[str, np.array]:
    to_numpy = lambda arrays: [np.array(a) for a in arrays]
    return {k:np.array(to_numpy(data[k])) for k in data}


def split_datasets(splits: int, data_set: t.Dict[str, list]) -> t.List[DataGroup]:
    data_set: t.Dict[str, np.array] = convert_to_numpy(data_set)
    for intent in data_set.keys():
        intent_masks: t.List[np.array] = get_masks(split, data_set[intent])
        
        

In [ ]:
def get_spliting_idx(splits, array) -> np.array:
    length = len(array)
    elements = length // splits
    rest = length % splits
    split_idx = np.full(splits, elements)
    split_idx[0] = split_idx[0] + rest
    return split_idx
    
def split_datasets(splits: int, data_set: dict):
    fragments = list()
    data = convert_to_numpy(data_set)
    splits_index: dict = {k:get_spliting_idx(splits, data[k]) for k in data}
    for intent in data_set.keys():
        split_idx = splits_index[intent]
        fragment = dict()
        
        
        fragments.append(fragment)

In [2]:

merged_matrix_repr = MergedMatrixRepresentation(processor.data)

[nltk_data] Error loading french: Package 'french' not found in index


./training.md


In [12]:
processor.data

{'welcome': [['salut'],
  ['coucou'],
  ['bonjour'],
  ['yo'],
  ['yop'],
  ['hello'],
  ['ça', 'va'],
  ['bijour'],
  ['kikou'],
  ['hi'],
  ['bien', 'quoi'],
  ['bienvenu'],
  ['réveil'],
  ['hep'],
  ['hey'],
  ['allo']],
 'search_movie': [['cherch', 'film'],
  ['don', 'idé', 'film'],
  ['aim', 'savoir', 'quel', 'film', 'voir', 'soir'],
  ['cherch', 'film', 'voir'],
  ['veux', 'regard', 'film'],
  ['don', 'film'],
  ['idé', 'film'],
  ['recherch', 'film'],
  ['sais', 'quoi', 'regard', 'comm', 'film'],
  ['quel', 'film', 'voir', 'moment'],
  ['besoin', 'idé', 'film', 'voir'],
  ['veux', 'voir', 'film', 'cet', 'anné'],
  ['film', 'sort', 'a', 'longtemp'],
  ['film', 'sort', 'cet', 'anné'],
  ['idé', 'truc', 'voir'],
  ['film', 'voir'],
  ['sais', 'quoi', 'regard', 'comm', 'film', 'soir'],
  ['besoin', 'idé', 'film'],
  ['idé', 'film', 'voir'],
  ['quel', 'film', 'peux', 'voir', 'soir'],
  ['sais', 'quoi', 'mat', 'comm', 'film'],
  ['matt', 'film'],
  ['regard', 'bon', 'film'],
  ['don

In [3]:
classifier = NaiveBayseTfIdfClassifier()
classifier_processor = ClassificationProcessor(classifier, merged_matrix_repr.data)
classifier_processor.train()
classifier.tf_idf

{'welcome': array([0.09873647, 0.09873647, 0.11755085, 0.11755085, 0.11755085,
        0.09873647, 0.11755085, 0.11755085, 0.11755085, 0.11755085,
        0.11755085, 0.09873647, 0.06110773, 0.11755085, 0.11755085,
        0.11755085, 0.11755085, 0.11755085, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.   

In [4]:
print(classifier.tf_idf['welcome'][64])
print(classifier.tf_idf['search_movie'][64])
print(classifier.tf_idf['meteo'][64])

0.0
0.0
0.4924348283103219


In [11]:
message = "je cherche un ciné"
message = processor.process_sentence(message)
message = merged_matrix_repr.process_new_data(message)
res = classifier_processor.predict(message)
print(message)
print(merged_matrix_repr.index.loc[merged_matrix_repr.index['lem'] == 't-il'])
utils.get_random_message(processor.responses[res[0]])

search_movie 0.1935483870967742
search_cinema 0.6470588235294118
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
Empty DataFrame
Columns: [lem, index]
Index: []


'va sur google !'

In [6]:
idx = merged_matrix_repr.index.loc[message == 1, 'index'].values
print(classifier.tf_idf['welcome'][idx])
print(classifier.tf_idf['search_movie'][idx])
print(classifier.tf_idf['meteo'][idx])
0.32590787 * 0.4262512 * 0.49243483

[0. 0. 0.]
[0.09461841 0.         0.        ]
[0.32590787 0.4262512  0.49243483]


0.0684083673568854

In [7]:
classifier = NaiveBayseClassifier()
classifier_processor = ClassificationProcessor(classifier, merged_matrix_repr.data)
classifier_processor.train()

In [8]:
message = "quel temps fait-il ?"
message = processor.process_sentence(message)
message = merged_matrix_repr.process_new_data(message)
res = classifier_processor.predict(message)
utils.get_random_message(processor.responses[res[0]])

search_movie 0.0967741935483871
search_cinema 0.058823529411764705
meteo 1.0833333333333333


"il fait moche ! Non, je déconne ! J'en sais rien !"

In [9]:
merged_matrix_repr.index.loc[60:, 'lem']

60    aujourd
61        hui
62      trouv
63       temp
64       fait
65     demain
66      pleut
67       beau
68     semain
69      météo
70       jour
71        met
72      froid
73        fer
74      chaud
Name: lem, dtype: object